In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

#대만 다트 접속해서 월별 매출 가져옴


# 년도별 월별 데이터를 가져오는 함수
def giga_monthly(year):
    
    url = 'https://emops.twse.com.tw/server-java/t05st10_ifrs_e'
    session = requests.Session()
    
    # 빈 데이터프레임 생성
    df_year = pd.DataFrame()

    # 1월부터 12월까지 데이터 수집
    for month in range(1, 13):
        time.sleep(3)
        # POST 요청에 사용할 데이터 설정
        data = {
            'co_id': '2377',   # 회사 ID
            'TYPEK': 'sii',    # 유형
            'step': '31',      # 스텝 (여기선 테이블 데이터를 가져오기 위한 스텝 설정)
            'YEAR': str(year), # 연도
            'MONTH': str(month) # 월
        }

        # POST 요청을 통해 서버에 데이터를 제출
        response = session.post(url, data=data)

        # BeautifulSoup을 이용해 HTML 파싱
        soup = BeautifulSoup(response.text, 'html.parser')

        # 'This Month' 수익 추출
        this_month_row = soup.find('td', string="This Month")
        if this_month_row:
            revenue = this_month_row.find_next('td').text.strip().replace(',', '')
            
            # 'Increase or Decrease Percentage' 추출
            percentage_row = soup.find('td', string="Increase or Decrease Percentage")
            percentage = percentage_row.find_next('td').text.strip()

            # 데이터를 누적
            data = {
                'Year': [year],
                'Month': [month],
                'Revenue': [int(revenue)],  # 수익은 정수로 변환
                'Increase/Decrease (%)': [float(percentage)]  # 증감률은 실수로 변환
            }

            df = pd.DataFrame(data)
            df_year = pd.concat([df_year, df], ignore_index=True)

        else:
            print(f"{year}-{month}: 데이터를 찾을 수 없습니다.")
    
    return df_year

# 함수 예시 사용
df_2023 = giga_monthly(2023)
df_2024 = giga_monthly(2024)

# 출력 결과 확인
print(df_2023)
print(df_2024)


2024-8: 데이터를 찾을 수 없습니다.
2024-9: 데이터를 찾을 수 없습니다.
2024-10: 데이터를 찾을 수 없습니다.
2024-11: 데이터를 찾을 수 없습니다.
2024-12: 데이터를 찾을 수 없습니다.
    Year  Month   Revenue  Increase/Decrease (%)
0   2023      1  13156226                 -35.47
1   2023      2  14111271                  -5.10
2   2023      3  16333250                  -8.15
3   2023      4  13109949                 -16.01
4   2023      5  13859134                 -12.89
5   2023      6  14322708                 -13.18
6   2023      7  13914288                   6.94
7   2023      8  16893813                  34.46
8   2023      9  18110067                  39.41
9   2023     10  19147888                  40.24
10  2023     11  18064202                  21.67
11  2023     12  11942671                  -2.82
   Year  Month   Revenue  Increase/Decrease (%)
0  2024      1  16392833                  24.60
1  2024      2  14819035                   5.02
2  2024      3  16414666                   0.50
3  2024      4  16405977                  25.14
